In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, dim, ffn_dim, dropout):
        super().__init__()
        self.fc1 = nn.Linear(dim, ffn_dim)
        self.fc2 = nn.Linear(ffn_dim, dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor):
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, ffn_dim, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(dim)
        self.ffn = FeedForward(dim, ffn_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None, return_attn: bool = False):
        # Self-attention with residual connection
        x_norm = self.ln1(x)
        attn_out, attn_weights = self.attn(x_norm, x_norm, x_norm, attn_mask=mask)
        x = x + self.dropout(attn_out)
        # Feed-forward network with residual connection
        x = x + self.dropout(self.ffn(self.ln2(x)))
        if return_attn:
            return x, attn_weights
        return x

class ClassicTransformer(nn.Module):
    def __init__(self, vocab_size, max_seq_len, dim, num_layers, num_heads, ffn_dim, dropout):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, dim)
        self.pos_embedding = nn.Embedding(max_seq_len, dim)
        self.layers = nn.ModuleList([
            TransformerBlock(dim, num_heads, ffn_dim, dropout) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(dim)
        self.output = nn.Linear(dim, vocab_size, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.max_seq_len = max_seq_len
        
    def forward(self, x: torch.Tensor, return_attn: bool = False):
        # x: [B, L]
        B, L = x.shape
        pos_ids = torch.arange(0, L, device=x.device).unsqueeze(0).expand(B, L)
        # Sum token and positional embeddings
        h = self.token_embedding(x) + self.pos_embedding(pos_ids)
        h = self.dropout(h)
        
        # Causal mask (upper triangular with -inf)
        if L > 1:
            mask = torch.triu(torch.full((L, L), float('-inf'), device=x.device), diagonal=1)
        else:
            mask = None
        
        attn_maps = []
        for layer in self.layers:
            if return_attn:
                h, attn_map = layer(h, mask, return_attn=True)
                attn_maps.append(attn_map)
            else:
                h = layer(h, mask)
            
        h = self.ln_f(h)
        logits = self.output(h)
        if return_attn:
            return logits, attn_maps
        return logits, None


In [2]:
# Hyperparameters for Transformer
vocab_size = 65
SEQUENCE_LENGTH = 64
NUM_LAYERS = 4
NUM_HEADS = 8
FFN_DIM = EMBEDDING_DIM = 256
DROPOUT = 0.1
device = 'cuda'

In [3]:
classic = ClassicTransformer(vocab_size, SEQUENCE_LENGTH, EMBEDDING_DIM, NUM_LAYERS, NUM_HEADS, FFN_DIM, DROPOUT).to(device)
classic

ClassicTransformer(
  (token_embedding): Embedding(65, 256)
  (pos_embedding): Embedding(64, 256)
  (layers): ModuleList(
    (0-3): 4 x TransformerBlock(
      (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (ffn): FeedForward(
        (fc1): Linear(in_features=256, out_features=256, bias=True)
        (fc2): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (ln_f): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (output): Linear(in_features=256, out_features=65, bias=False)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [4]:
from torchinfo import summary

# Describe the model
summary(classic, input_size=(512, SEQUENCE_LENGTH), dtypes=[torch.long])

Layer (type:depth-idx)                   Output Shape              Param #
ClassicTransformer                       [512, 64, 65]             --
├─Embedding: 1-1                         [512, 64, 256]            16,640
├─Embedding: 1-2                         [512, 64, 256]            16,384
├─Dropout: 1-3                           [512, 64, 256]            --
├─ModuleList: 1-4                        --                        --
│    └─TransformerBlock: 2-1             [512, 64, 256]            --
│    │    └─LayerNorm: 3-1               [512, 64, 256]            512
│    │    └─MultiheadAttention: 3-2      [512, 64, 256]            263,168
│    │    └─Dropout: 3-3                 [512, 64, 256]            --
│    │    └─LayerNorm: 3-4               [512, 64, 256]            512
│    │    └─FeedForward: 3-5             [512, 64, 256]            131,584
│    │    └─Dropout: 3-6                 [512, 64, 256]            --
│    └─TransformerBlock: 2-2             [512, 64, 256]          